### LCEL Deepdive

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

'Why did the ice cream truck break down?\n\nBecause it had a rocky road! 🍦😄'

In [3]:
prompt.invoke({"topic": "ice cream"})

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream', additional_kwargs={}, response_metadata={})])

In [4]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

AIMessage(content='Why did the ice cream cone become a motivational speaker?  \n\nBecause it always knew how to scoop people up!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 15, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BcYcJgFbw9PjFzIcPJWmebNCJxB4I', 'finish_reason': 'stop', 'logprobs': None}, id='run-19f6a0f3-0aa4-4c5b-9654-21059bb544f7-0', usage_metadata={'input_tokens': 15, 'output_tokens': 22, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.messages import AIMessage

ai_msg = AIMessage(content='Why did the ice cream truck break down? It had too many "scoops"!')

output_parser.invoke(ai_msg)

'Why did the ice cream truck break down? It had too many "scoops"!'

### Operator Overloading

In [6]:
print(2+4)

6


In [7]:
result = (2).__add__(4)

print(result)

6


In [8]:
class StupidAdder:
    def __init__(self, number):
        self.number = number

    def __add__(self, other):
        return StupidAdder(self.number + other.number + 42)

    def __str__(self):
        return str(self.number)

In [9]:
first = StupidAdder(5)
second = StupidAdder(10)

print(first + second)

57


### What is this "|" in Python?

In [10]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        pass

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



In [11]:
class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

In [12]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

In [13]:
result = chain.invoke(10)
print(result)

AddTen:  10
Multiply by 2:  20
Convert to string:  40
Result: 40


### Runnables from LangChain

In [14]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [15]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

'hello'

In [16]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [17]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

'HELLO'

In [18]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [19]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [20]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'},
 'y': {'input': 'hello', 'input2': 'goodbye'}}

In [21]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [22]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'y': 'goodbye'}

### Nested chains - now it gets more complicated!

In [23]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [24]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

In [25]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': 'HELLO', 'y': 'goodbye'}

In [26]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(_):
    return 100

def multiply(input):
    return input * 10

In [27]:
chain.invoke({"input": "hello", "input2": "goodbye"})

{'x': {'input': 'hello', 'input2': 'goodbye'}}

In [28]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

In [29]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

{'x': {'input': 'hello', 'input2': 'goodbye'}, 'extra': 100}


### Combine multiple chains

In [30]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [31]:
new_chain.invoke({"extra": "test"})

'TEST'

In [32]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

'100'